In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.system import System
from pof.component import Component
from pof.failure_mode import FailureMode
import pof.demo as demo
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.data.asset_data import SimpleFleet

In [2]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

paths = Paths()

# Population Data
file_path = paths.input_path + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR);

In [3]:
aml = AssetModelLoader(paths.demo_path + os.sep + "Asset Model.xlsx")
sys_data = aml.load(paths.demo_path + os.sep + "Asset Model.xlsx")
sys = System.from_dict(sys_data["overhead_network"])
sys.fleet_data = sfd

In [4]:
t_end = 100
n_iterations = 100

In [5]:
# Turn failure modes on & off

components_on = ["pole"]
failure_modes_on = ["termites", "fungal decay | internal"]
tasks_off = ['pole_saver_rod']

for comp in sys.comp.values():
    if comp.name not in components_on:
        comp.active = False
    else:
        comp.active = True
    for fm in comp.fm.values():
        if fm.name not in failure_modes_on:
            fm.active = False
        else:
            fm.active = True

        for task in fm.tasks.values():
            if task.name in tasks_off:
                task.active = False
            else:
                task.active = True

In [6]:
sys.comp["pole"].fm['fungal decay | internal'].tasks['inspection_groundline'].p_effective = 1

In [7]:
sys.mc_timeline(t_end=t_end, n_iterations=n_iterations)

100%|██████████| 100/100 [00:00<00:00, 172.11it/s]


In [8]:
sys.expected_risk_cost_df()

sys.df_erc[sys.df_erc["quantity"] > 0];

sys.df_erc;

,failure_mode,task,active,time,quantity,cost,cost_cumulative,cost_annual,quantity_cumulative,quantity_annual,comp
28,termites,risk,True,0,0,0,0,NaN,0,NaN,pole
29,fungal decay | external,risk,False,0,0,0,0,NaN,0,NaN,pole
30,fungal decay | internal,risk,True,0,0,0,0,NaN,0,NaN,pole
31,lightning,risk,False,0,0,0,0,NaN,0,NaN,pole
32,weathering,risk,False,0,0,0,0,NaN,0,NaN,pole
33,fire_damage,risk,False,0,0,0,0,NaN,0,NaN,pole
34,impact,risk,False,0,0,0,0,NaN,0,NaN,pole
14,termites,conditional_failure,True,0,0,0,0,NaN,0,NaN,pole
15,fungal decay | external,conditional_failure,False,0,0,0,0,NaN,0,NaN,pole
16,fungal decay | internal,conditional_failure,True,0,0,0,0,NaN,0,NaN,pole


In [9]:
comps_to_check = ["pole"]
fms_to_check = ["termites", "fungal decay | internal"]
tasks_to_check = ["conditional_failure", "functional_failure"]

In [10]:
dict_index = dict()

for comp_name, comp in sys.comp.items():
    if comp_name in comps_to_check:
        for fm_name, fm in comp.fm.items():
            if fm_name in fms_to_check:
                for index, timeline in fm._timelines.items():
                    for task_name in tasks_to_check:
                        if 0 in timeline[task_name]: 
                            dict_index[index] = {"fm":fm_name, "task":task_name}

dict_index;

{4: {'fm': 'termites', 'task': 'functional_failure'},
 5: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 15: {'fm': 'termites', 'task': 'functional_failure'},
 16: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 18: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 19: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 23: {'fm': 'termites', 'task': 'functional_failure'},
 25: {'fm': 'termites', 'task': 'functional_failure'},
 26: {'fm': 'termites', 'task': 'functional_failure'},
 28: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 39: {'fm': 'termites', 'task': 'functional_failure'},
 42: {'fm': 'termites', 'task': 'functional_failure'},
 43: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 50: {'fm': 'termites', 'task': 'functional_failure'},
 52: {'fm': 'fungal decay | internal', 'task': 'functional_failure'},
 56: {'fm': 'termites', 'task': 'functional_failure'},
 59: {'fm': 'term

In [11]:
sys.comp["pole"].fm["fungal decay | internal"]._timelines[97];

{'time': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100]),
 'initiation': array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  T